# Multimethod polymorphism

## Polymorphism and its types

### Ad-hoc polymorphism

"closed-dispatch" polymorphism

In [1]:
; A function that ckecks the type of 'thing' argument 
; to call different functions based on the type

(defn ad-hoc-type-namer [thing]
    (condp = (type thing) 
        java.lang.String "string" 
        clojure.lang.PersistentVector "vector"))


#'user/ad-hoc-type-namer

In [2]:
; Passing a string parameter

(ad-hoc-type-namer "I'm a string")

"string"

In [3]:
; Passing a vector parameter

(ad-hoc-type-namer [])

"vector"

In [4]:
; If there’s a type this function doesn’t know how to handle, an exception is thrown

(ad-hoc-type-namer {}) 

Execution error (IllegalArgumentException) at user/ad-hoc-type-namer (REPL:5).
No matching clause: class clojure.lang.PersistentArrayMap


class java.lang.IllegalArgumentException: 

"open-dispatch" polymorphism

In [5]:
; Pull type implementations out into a separate map

(def type-namer-implementations 
    {java.lang.String (fn [thing] "string")
     clojure.lang.PersistentVector (fn [thing] "vector")})


#'user/type-namer-implementations

In [6]:
; A function that ckecks the type of 'thing' argument 
; to call different functions based on the type, 
; without a fixed set of types to ckeck
 
(defn open-ad-hoc-type-namer [thing]
    (let [dispatch-value (type thing)] 
        ; Use a dispatch value as key to implementation map
        (if-let [implementation (get type-namer-implementations dispatch-value)] 
            (implementation thing) 
            (throw (IllegalArgumentException. (str "No implementation found for " dispatch-value))))))


#'user/open-ad-hoc-type-namer

In [7]:
; Passing a parameter of string type

(open-ad-hoc-type-namer "I'm a string") 

"string"

In [8]:
; Passing a parameter of vector type

(open-ad-hoc-type-namer []) 

"vector"

In [9]:
; Passing a parameter of map type (it will throw an exception)

(open-ad-hoc-type-namer {}) 

Execution error (IllegalArgumentException) at user/open-ad-hoc-type-namer (REPL:10).
No implementation found for class clojure.lang.PersistentArrayMap


class java.lang.IllegalArgumentException: 

In [10]:
; Adding a 'map' type to a the map of types

(def type-namer-implementations 
    (assoc type-namer-implementations clojure.lang.PersistentArrayMap (fn [thing] "map")))


#'user/type-namer-implementations

In [11]:
; Passing a parameter of map type again (it won't throw an exception now)

(open-ad-hoc-type-namer {})

"map"

### Subtype polymorphism

Checking types with ad-hoc polymorphism

In [12]:
; A function that checks if its argument is map-like

(defn map-type-namer [thing] 
    (condp = (type thing)
        clojure.lang.PersistentArrayMap "map"
        clojure.lang.PersistentHashMap "map")) ; Notice the code duplication

#'user/map-type-namer

In [13]:
; Passing a parameter of hash-map type (recognized)

(map-type-namer (hash-map))

"map"

In [14]:
; Passing a parameter of array-map type (recognized)

(map-type-namer (array-map))

"map"

In [15]:
; Passing a parameter of sorted-map type (unrecognized)

(map-type-namer (sorted-map)) 

Execution error (IllegalArgumentException) at user/map-type-namer (REPL:4).
No matching clause: class clojure.lang.PersistentTreeMap


class java.lang.IllegalArgumentException: 

Checking types with subtype polymorphism

In [16]:
(defn subtyping-map-type-namer [thing] 
    (cond
        (instance? clojure.lang.APersistentMap thing) "map" ; APersistent-Map is Java superclass of all map-like things in Clojure
         :else (throw (IllegalArgumentException. (str "No implementation found for ") (type thing)))))


#'user/subtyping-map-type-namer

In [17]:
; Passing a parameter of hash-map type (recognized)

(subtyping-map-type-namer (hash-map))

"map"

In [18]:
; Passing a parameter of array-map type (recognized)

(subtyping-map-type-namer (array-map))

"map"

In [19]:
; Passing a parameter of sorted-map type (recognized now)

(subtyping-map-type-namer (sorted-map)) 

"map"

## Polymorphism using multimethods

Implementing a tracking expense service without multimethods 

In [20]:
; A mapping for a sample user

(def example-user {:login "rob" 
                   :referrer "mint.com" 
                   :salary 100000})


#'user/example-user

In [21]:
; A function that calculates the fee depending of the referrer type

(defn fee-amount [percentage user]
    (with-precision 16 :rounding HALF_EVEN 
        ; Using BigDecimal when dealing with money
        (* 0.01M percentage (:salary user))))

#'user/fee-amount

In [22]:
; A function that performs close dispatch based on the referrer type

(defn affiliate-fee [user]
    (case (:referrer user)
        "google.com" (fee-amount 0.01M user)
        "mint.com" (fee-amount 0.03M user)
        (fee-amount 0.02M user)))


#'user/affiliate-fee

In [23]:
; Testing the last function with the example-user map

(affiliate-fee example-user)


30.0000M

Implementing a tracking expense service with multimethods 

In [24]:
; A general form of the defmulti macro

"""
(defmulti name docstring? attr-map? dispatch-fn & options)
"""

""

In [25]:
; The defmulti returns a dispatch value 
; with which to find an implementation.

(defmulti affiliate-fee 
    (fn [user] (:referrer user)))


#'user/affiliate-fee

In [26]:
; Defining an implementation for the "mint.com" dispatch value

(defmethod affiliate-fee "mint.com" [user] 
    (fee-amount 0.03M user))

#multifn[affiliate-fee 0x63953eae]

In [27]:
; Defining an implementation for the "google.com" dispatch value

(defmethod affiliate-fee "google.com" [user]
    (fee-amount 0.01M user))

#multifn[affiliate-fee 0x63953eae]

In [28]:
; Defining an implementation for a default dispatch value

(defmethod affiliate-fee :default [user] 
    (fee-amount 0.02M user))

#multifn[affiliate-fee 0x63953eae]

In [29]:
; Testing the first method with the example-user map

(affiliate-fee example-user)

30.0000M

Using multimethods with custom defaults

In [30]:
; Redefining the default case (it won't take effect)

(defmulti affiliate-fee :referrer :default "*")

nil

In [31]:
; Removing the affiliate-fee multimethod from the user namespace

(ns-unmap 'user 'affiliate-fee)

nil

In [32]:
; Redefining the default case (it will take effect now)

(defmulti affiliate-fee :referrer :default "*") 

#'user/affiliate-fee

In [33]:
; Defining the new default case method

(defmethod affiliate-fee "*" [user] 
    (fee-amount 0.02M user))

#multifn[affiliate-fee 0x51b7ab75]

In [34]:
(affiliate-fee example-user)

20.0000M

In [35]:
; A general form of the defmethod macro

"""
(defmethod multifn dispatch-value & fn-tail)
"""


""

In [36]:
; The defmethod macro can also be used with multiple arities

"""
(defmethod my-many-arity-multi :default
    ([] \"no arguments\")
    ([x] \"one argument\")
    ([x & etc] \"many arguments\"))
"""

""

Accessing multimethod map items

In [37]:
; Defining again the method for "mint.com"

(defmethod affiliate-fee "mint.com" [user] 
    (fee-amount 0.03M user))

#multifn[affiliate-fee 0x51b7ab75]

In [38]:
; Defining again the method for "google.com"

(defmethod affiliate-fee "google.com" [user]
    (fee-amount 0.01M user))

#multifn[affiliate-fee 0x51b7ab75]

In [39]:
; Returning all the methods associated to "affiliate-fee"

(methods affiliate-fee) 

{"mint.com" #function[user/eval4178/fn--4179], "*" #function[user/eval4172/fn--4173], "google.com" #function[user/eval4182/fn--4183]}

In [40]:
; Returning the method associated to "mint.com"

(get-method affiliate-fee "mint.com") 

#function[user/eval4178/fn--4179]

In [41]:
; No specific method associated to "example.org"

(get (methods affiliate-fee) "example.org")

nil

In [42]:
; The "*" (default) implementation

(get-method affiliate-fee "example.org") 

#function[user/eval4172/fn--4173]

In [43]:
; The entries in the dispatch map can be called as normal functions.

((get-method affiliate-fee "mint.com") example-user) 

30.0000M